# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly_express as px
from pathlib import Path
from panel.interact import interact


# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [ ]:
# Read the Mapbox API key
mapbox_token = os.getenv("~~~~~")

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    sfo_years = sfo_data.groupby("year")
    mean_units_per_year = sfo_years["housing_units"].mean()
    mean_units_per_year.plot(kind = "bar", ylim = (369000, 390000), title = "Average Housing Units per Year")

    return 


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    avg_gross_rent = sfo_years["gross_rent"].mean()
    avg_sales_price_sqft = sfo_years["sale_price_sqr_foot"].mean()
    gross_rent_sale_price = sfo_years["sale_price_sqr_foot", "gross_rent"].mean()
    avg_gross_rent.plot(title = "Average Gross Rent Per Year in San Francisco")

    return

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    avg_sales_price_sqft.plot(title = "Average Sales Price per Square Foot in San Francisco")

    return

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    
    year_neighborhood_avgprice = sfo_data.filter(["year","neighborhood","sale_price_sqr_foot"]).groupby([sfo_data.index,sfo_data["neighborhood"]]).mean()
    year_neighborhood_avgprice.hvplot.line(x="year",y="sale_price_sqr_foot",groupby="neighborhood")

    return


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    ten_most_expensive_neighborhoods = sfo_data.filter(["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"]).groupby([sfo_data['neighborhood']]).mean()
    ten_most_expensive_neighborhoods = ten_most_expensive_neighborhoods.sort_values(by='sale_price_sqr_foot', ascending=False).head(10)
    top_ten_chart = ten_most_expensive_neighborhoods.drop(["housing_units", "gross_rent"], axis = 1)
    top_ten_chart.plot.bar(title="Ten Most Expensive Neighborhoods in San Francisco", figsize = (13,6))

    return top_most_expensive_neighborhoods

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    sales_price_neighborhood = sfo_data.groupby([sfo_data["neighborhood"]]).mean()
    sales_price_neighborhood = sales_price_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    parallel_coordinates_plot = px.parallel_coordinates(
    sales_price_neighborhood,
    dimensions=["sale_price_sqr_foot","housing_units","gross_rent"],
    color = "sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
     labels={
        "sale_price_sqr_foot": "Sale Price per Square Foot",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"    
        }
    )

    return

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    parallel_categories_plot = px.parallel_categories(
        sales_price_neighborhood.reset_index(),
        dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "sale_price_sqr_foot": "Sale Price per Square Foot",
            "housing_units": "Housing Units",
            "gross_rent": "Gross Rent"
        }
    )
    return

def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    data_prep_df = sfo_data.groupby([sfo_data["neighborhood"]]).mean()
    sfo_price_and_location = pd.concat([data_prep_df, df_neighborhood_locations],axis=1, join="inner")

    px.scatter_mapbox(
    sfo_price_and_location.reset_index(),
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire, 
    zoom=11,
    text="neighborhood"
    )
    return

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# YOUR CODE HERE!
tabs = pn.Tabs(
    (" 10 Most Expensive Neighborhoods in San Francisco ",top_most_expensive_neighborhoods()),
    (" Top 10 Parallel Coordinates ",housing_units_per_year()),
    (" Top 10 Parallel Categories ",parallel_categories())
    ("Explore the Map",interact(neighborhood_map,number=30)),
    ("Housing Units per Year",housing_units_per_year()),
    ("Avarage Gross Rent per Year",average_gross_rent()),
    ("Avarage Sales Price/Sqft per Year",average_sales_price())
    
)

## Serve the Panel Dashboard

In [ ]:
panel.servable()